# Schema Validation, Advanced Queries and more

In [3]:
import pyarrow
from pymongoarrow.api import Schema
from pymongoarrow.monkey import patch_all
import pymongoarrow as pma
from datetime import datetime as dt
import pprint
import json
import pandas as pd
from pymongo import MongoClient
from bson import ObjectId


client = MongoClient('mongodb://localhost:27017/&authSource=admin')

client.list_database_names()
production = client.production

In [4]:
# Schema validation is essentially a way of creating some type of structure in your MDB database. 
# When you insert a document into a collection, there is nothing that enforces what kind of fields are being input
# Also when you insert a collection, there is nothing that enforces what kind of columns are necessary. 

#Sometimes you want to have enforcement, and this is where schema validation comes in. The validator witin the schema allows 
# you to pass in the required params. 


def create_book_collection():

    book_validator = {
                  "$jsonSchema": {
                    "bsonType": "object",
                    "required": ["title", "authors", "publish_date", "type", "copies"],
                    "properties": {
                      "title": {
                        "bsonType": "string",
                        "description": "Must be a string and is required."
                      },
                      "authors": {
                        "bsonType": "array",
                        "items": {
                          "bsonType": "objectId",
                          "description": "Must be an objectId and is required."
                        }
                      },
                      "publish_date": {
                        "bsonType": "date",
                        "description": "Must be a date and is required."
                      },
                      "type": {
                        "enum": ["Fiction", "Non-Fiction"],
                        "description": "Can only be one of the enum values and is required."
                      },
                      "copies": {
                        "bsonType": "int",
                        "minimum": 0,
                        "description": "Must be an integer greater than or equal to 0 and is required."
                      }
                    }
                  }
                }


    # create the book collection, and insert the book_validator for that specific collectiom.
    # You can also alter the validation action & level. 
    try:
        production.create_collection('book')
    except Exception as e:
        print(e)


    production.command('collMod', 'book', validator=book_validator)
    
    
    # Create the validation schema for the author collection, and creates the author collection if it does not exist

def create_author_collection():
    author_validator = {
        '$jsonSchema': {
            'bsonType': 'object',
            'required': ['first_name', 'last_name', 'date_of_birth'],
            'properties': {
                'first_name': {
                    'bsonType': 'string',
                    'description': 'must be a string and is required'
                },
                'last_name': {
                    'bsonType': 'string',
                    'description': 'must be a string and is required'
                },
                'date_of_birth': {
                    'bsonType': 'date',
                    'description': 'must be a date and is required'
                }
            }
        }
    }
    
    
    try:
        production.create_collection('author')
    except Exception as e:
        print(e)
        
    production.command('collMod', 'author', validator=author_validator)
    
create_author_collection()
    

collection author already exists


# Create Data, Making Relations Between Collections

In [5]:
# We have an array of authors that are referencing object IDs of the authors from the authors collection 

# By putting the authors in a seperate collection, rather than embedding them....
# When you have a 1 to 1 relationship it is totally fine to store this as an embedded document inside of the person. 
# However, if there is an address that is associated with many people, than it makes sense to take that docuement and
# store it as a seperate collection. Then make a reference to that address from all of the docs that reference. 

# The reason why it would be more efficient this way is because it would take up less space, and would not have to 
# duplicate it over and over. Also means when you make a modification to the address, there is one singular place, vs 
# changing it in multiple docuements/locations. 

# It can be very time consuming and slow down your server, if you are grabbing every reference for a specific document
#every time you retrieve that doc. 


def create_data():
    
    authors = [
        {
            'first_name': 'Tim',
            'last_name': 'Ruscica',
            'date_of_birth': dt(2000, 7, 20)
        },
        {
             'first_name': 'George',
            'last_name': 'Orwell',
            'date_of_birth': dt(1903, 6, 25)
        },
        {
             'first_name': 'Herman',
            'last_name': 'Melville',
            'date_of_birth': dt(1819, 8, 1)
        },
        {
             'first_name': 'F. Scott',
            'last_name': 'Fitzgerald',
            'date_of_birth': dt(1896, 9, 24)
        }
    ]
    
    author_collection = production.author
    #get inserted_ids through the insert
    authors = author_collection.insert_many(authors).inserted_ids
    
    
    books = [
        {
            'title': 'MDB Advanced Tutorial',
            'authors': [authors[0]],
            'publish_date': dt.today(),
            'type': 'Non-Fiction',
            'copies': 5
        },
        {
            'title': 'Python For Dummies',
            'authors': [authors[0]],
            'publish_date': dt(2023, 5, 15),
            'type': 'Non-Fiction',
            'copies': 50
        },
        {
            'title': '1984',
            'authors': [authors[1]],
            'publish_date': dt(1949, 6, 8),
            'type': 'Fiction',
            'copies': 75
        },
        {
            'title': 'The Great Gatsby',
            'authors': [authors[3]],
            'publish_date': dt(1851, 9, 24),
            'type': 'Fiction',
            'copies': 5
        }
    ]

    book_collection = production.book
    book_collection.insert_many(books)
    
    
create_data()
    
    

# MDB Aggregations

In [6]:
# Retrieve all of the books that contain the letter 'A'
# Regex says that we want a regular expression with at least 1. 

books_containing_a = production.book.find({'title': {'$regex': 'a{1}'}})

# pprint.pprint(list(books_containing_a))


# Allows us to aggregate/chain multiple operators together. 
# The localField is our field in authors, foreignField is what we are joining on in authors.
# As is the field that we are adding that contains all of the books.  

# The field joins the book collection, on the authors collection using the 

authors_and_books = production.author.aggregate([
    {
        '$lookup': {
            'from': 'book', 
            'localField': '_id', 
            'foreignField': 'authors', 
            'as': 'books'
        }
    },
    {
        '$addFields': {
            'total_books': {'$size': '$books'}
        }
    },
    {
       '$project': {'first_name': 1, 'last_name': 1, 'total_books': 1, '_id': 0}
    }
    ]
    )

pprint.pprint(list(authors_and_books))

[{'first_name': 'Tim', 'last_name': 'Ruscica', 'total_books': 2},
 {'first_name': 'George', 'last_name': 'Orwell', 'total_books': 1},
 {'first_name': 'Herman', 'last_name': 'Melville', 'total_books': 0},
 {'first_name': 'F. Scott', 'last_name': 'Fitzgerald', 'total_books': 1},
 {'first_name': 'Tim', 'last_name': 'Ruscica', 'total_books': 2},
 {'first_name': 'George', 'last_name': 'Orwell', 'total_books': 1},
 {'first_name': 'Herman', 'last_name': 'Melville', 'total_books': 0},
 {'first_name': 'F. Scott', 'last_name': 'Fitzgerald', 'total_books': 1},
 {'first_name': 'Tim', 'last_name': 'Ruscica', 'total_books': 2},
 {'first_name': 'George', 'last_name': 'Orwell', 'total_books': 1},
 {'first_name': 'Herman', 'last_name': 'Melville', 'total_books': 0},
 {'first_name': 'F. Scott', 'last_name': 'Fitzgerald', 'total_books': 1}]


In [7]:
# Want to filter for authors with books that are between 100 & 150 years old 
# Need to calculate the age using the date field. 

# First join the books with the authors, loop through the authors and set the age of them. THen make our query for our range.

books_with_old_authors = production.book.aggregate([
    {
        '$lookup': {
            'from': 'author',
            'localField': 'authors',
            'foreignField': '_id',
            'as': 'authors'
        }
    },
    
    {'$set': {
        'authors': {
                '$map': {                       #We iterate over every element, and tell it what we want as the fields
                    'input': '$authors',        # by using the map operator. 
                    'in': {
                        'age': {
                            '$subtract': [
                                {'$year': '$$NOW'},
                                {'$year': '$$this.date_of_birth'}
                            ]
                        },
                        'first_name': '$$this.first_name',
                        'last_name': '$$this.last_name'
                    }
                }
            }
        }
    },
    {                                     # The join is made from the book collection to the author collection on the _id field
        '$match': {                        #the age field is created. Now we can filter for the proper range
            'authors.age': { 
                '$gte': 50,
                '$lte': 150
            }
        }
    },
        {
        '$sort': {
            'authors.age': 1
        }
    }
])

pprint.pprint(list(books_with_old_authors))


[{'_id': ObjectId('649f39168df97e2e1f6dc37b'),
  'authors': [{'age': 120, 'first_name': 'George', 'last_name': 'Orwell'}],
  'copies': 75,
  'publish_date': datetime.datetime(1949, 6, 8, 0, 0),
  'title': '1984',
  'type': 'Fiction'},
 {'_id': ObjectId('649fd07a4f1fb4c142736773'),
  'authors': [{'age': 120, 'first_name': 'George', 'last_name': 'Orwell'}],
  'copies': 75,
  'publish_date': datetime.datetime(1949, 6, 8, 0, 0),
  'title': '1984',
  'type': 'Fiction'},
 {'_id': ObjectId('649fd184e02a8fae2e0f37a7'),
  'authors': [{'age': 120, 'first_name': 'George', 'last_name': 'Orwell'}],
  'copies': 75,
  'publish_date': datetime.datetime(1949, 6, 8, 0, 0),
  'title': '1984',
  'type': 'Fiction'},
 {'_id': ObjectId('649f39168df97e2e1f6dc37c'),
  'authors': [{'age': 127,
               'first_name': 'F. Scott',
               'last_name': 'Fitzgerald'}],
  'copies': 5,
  'publish_date': datetime.datetime(1851, 9, 24, 0, 0),
  'title': 'The Great Gatsby',
  'type': 'Fiction'},
 {'_id': Obj

In [10]:
client = MongoClient('mongodb://localhost:27017/&authSource=admin')

client.list_database_names()
production = client.production

patch_all()

#specify schema, and load the data. Difficult to read in data from MDB, and this is the shortcut. 

author = Schema({'_id': ObjectId, 'first_name': pyarrow.string(), 
                 'last_name': pyarrow.string(), 'date_of_birth': dt})

df = production.author.find_pandas_all({}, schema=author)
print(df.head())

                        _id first_name   last_name date_of_birth
0  649f39168df97e2e1f6dc375        Tim     Ruscica    2000-07-20
1  649f39168df97e2e1f6dc376     George      Orwell    1903-06-25
2  649f39168df97e2e1f6dc377     Herman    Melville    1819-08-01
3  649f39168df97e2e1f6dc378   F. Scott  Fitzgerald    1896-09-24
4  649fd07a4f1fb4c14273676d        Tim     Ruscica    2000-07-20


In [11]:
arrow_table = production.author.find_arrow_all({}, schema=author)
ndarrays = production.author.find_numpy_all({}, schema=author)
print(ndarrays)

{'_id': array([b'd\x9f9\x16\x8d\xf9~.\x1fm\xc3u',
       b'd\x9f9\x16\x8d\xf9~.\x1fm\xc3v',
       b'd\x9f9\x16\x8d\xf9~.\x1fm\xc3w',
       b'd\x9f9\x16\x8d\xf9~.\x1fm\xc3x', b'd\x9f\xd0zO\x1f\xb4\xc1Bsgm',
       b'd\x9f\xd0zO\x1f\xb4\xc1Bsgn', b'd\x9f\xd0zO\x1f\xb4\xc1Bsgo',
       b'd\x9f\xd0zO\x1f\xb4\xc1Bsgp',
       b'd\x9f\xd1\x84\xe0*\x8f\xae.\x0f7\xa1',
       b'd\x9f\xd1\x84\xe0*\x8f\xae.\x0f7\xa2',
       b'd\x9f\xd1\x84\xe0*\x8f\xae.\x0f7\xa3',
       b'd\x9f\xd1\x84\xe0*\x8f\xae.\x0f7\xa4'], dtype=object), 'first_name': array(['Tim', 'George', 'Herman', 'F. Scott', 'Tim', 'George', 'Herman',
       'F. Scott', 'Tim', 'George', 'Herman', 'F. Scott'], dtype='<U8'), 'last_name': array(['Ruscica', 'Orwell', 'Melville', 'Fitzgerald', 'Ruscica', 'Orwell',
       'Melville', 'Fitzgerald', 'Ruscica', 'Orwell', 'Melville',
       'Fitzgerald'], dtype='<U10'), 'date_of_birth': array(['2000-07-20T00:00:00.000', '1903-06-25T00:00:00.000',
       '1819-08-01T00:00:00.000', '1896-09-24